## Getting Started

Welcome to Jupyter notebook.

Each block of text/code is contained in a cell. The active cell is the one where your cursor is active. You can run an active cell by pressing "Shift + Enter" while it is active.

After you run a cell, the output is shown below it. You can rerun a cell as many times as you'd like. Try it out below

In [ ]:
1 + 1

If that's working, we can get started.

First, you need to install the Skyrmions3D package on your computer. This will take a long time (15 minutes) the first time you do it because it needs to install all the other packages Skyrmions3D relies on. The second time you do this, it will take less time (1 minute).

Once it becomes a real "Package" this step will take no time.

In [ ]:
]add https://github.com/chrishalcrow/Skyrmions3D.jl.git

We'll also install the Package `Makie`. This will be used for plotting and for quaternionic algebra (plotting packages use quaternions for camera angles etc)

In [ ]:
]add Makie

Now that the `Skyrmions` and `Makie` packages are installed, we need to tell this notebook to use them. Again, this will take a little time (~1 minute)

In [ ]:
using Skyrmions
using Makie

That's the hard part done.

Let's make an initial Skyrme field.

In this code, your skyrmion object contains more than just the skyrme field. It also contains the grid you made the skyrmion on (and will contain the model parameters). So when you initialise the skyrmion you need to specify a grid. You can do this by running

my_skyrmion = Skyrmion( number_of_lattice_points, lattice_spacing  )

(note: for unequal numbers of points, just use e.g. Skyrmion( [30,30,60], [0.2,0.2,0.2] )  )

Like so:

In [ ]:
my_skyrmion = Skyrmion(40, 0.2)

That's a lot of output! You can tell Jupyter to not print the output of a cell by putting a semi-colon at the end of a line. Try putting ; at the end of the code above and re-run the code.

Now that we've made an empty Skyrme field, we need to fill it. One option is to use the Rational Map approx. For this, we need to specify a Rational Map and a profile funciton.

My implementation takes in two complex functions p(z), q(z) which are the numerator and denominantor of the rational map R(z) = p(z)/q(z) and a profile function f(r). Let's see an example:


In [ ]:
# A rational map and profile function

p4(z) = z^4 + 2.0*sqrt(3.0)*im*z^2 + 1.0
q4(z) = z^4 - 2.0*sqrt(3.0)*im*z^2 + 1.0
f4(r) = pi*exp( -(r.^3)./12.0 )

In [ ]:
makeRM!(my_skyrmion, f4, p4, q4)

A couple of notes: functions with an exclamation point(!) are modifying functions. So here, makeRM! is modifying the previously-intialised field "my_skyrmion"

We can find some properties of the newly created skyrmion:

In [ ]:
Energy(my_skyrmion)

In [ ]:
Baryon(my_skyrmion)

Note, the pion mass (in dimensionless units) is stored within your skyrmion. You can change it as follows:
(and then calculate the Energy to see the difference)

In [ ]:
my_skyrmion.mpi = 1.0
Energy(my_skyrmion)

## Plotting

Nice!

We can now plot it to take a look. Note: the first time you do this, Julia compiles the plotting functions. This can take some time (~1 minute), but will only happen the first time you plot.

Depending on your operating system, and some default options, many different things might happen right now. Most likely a static images will appear below, or **nothing** will appear.

In [ ]:
plot_baryon_density(my_skyrmion)

We can make a better plot by telling `Makie` exactly how we'd like to plot. I find that the best option is to create a new window, which allows for smooth interactive plots. To do this, we first tell `Makie` to _not_ plot `inline`, tell the `Skyrmions` package that we want an interactive plot and then we project the plot into the window using `display`

In [ ]:
Makie.inline!(false)
interactive_plot() 
display(plot_baryon_density(my_skyrmion))

If that worked... great! If not, please get in touch.

Now that we've told `Makie` and `Skyrmions` how we'd like to plot, we don't need to tell them again.

The baryon density plotting function can take in any arguments which are used in the `Axis3` method from Makie. So, for example, we can change the aspect ratio, add a title and get rid of the box:

In [ ]:
display(plot_baryon_density(my_skyrmion,iso_value=0.5, title="A B=4 skyrmion", aspect=(1.0,1.0,2.0), xlabel="the x-axis", xspinesvisible=false, yspinesvisible=false, zspinesvisible=false ) )

You can check out more options here: https://docs.makie.org/stable/examples/blocks/axis3/

You can even use LaTeX, if you add the LatexString package

In [ ]:
]add LaTeXStrings

In [ ]:
using LaTeXStrings

display(plot_baryon_density(my_skyrmion,iso_value=0.5,title=L"\text{A } B=4 \text{ skyrmion}" ) )

You can also export images of the plot using the `save` funtion. The `px_per_unit` controls the pixel density (and hence the quality and size of the exported file)

In [ ]:
save("B4_plot.png",plot_baryon_density(my_skyrmion,azimuth=0.5,elevation=0.5), px_per_unit=2)

## Transformations -- translations, rotations, isorotations and products

We can apply simple transformations to the skyrmions using translate_sk!, rotate_sk! and isorotate_sk!. These ! functions modifiy the initial skyrmion. If you'd like to make a brand new skyrmion, you should use the !-less version of the function.
    
Below are some examples:

In [ ]:
# Rotations and isorotations use "angle-axis" notation
# Note: we (currently) rotate around the origin, so it's best to rotate first.

rotate_sk!(my_skyrmion, pi/4, [1.0,1.0,0.0] )
translate_sk!(my_skyrmion,[1.0,-1.0,1.0])
isorotate_sk!(my_skyrmion, pi/2, [-2.0,1.0,1.0] )


display(plot_baryon_density(my_skyrmion,iso_value=0.5))

In [ ]:
# or we can make a new skyrmion from old (let's re-initialise the B=4 RM skyrmion first)

makeRM!(my_skyrmion, f4, p4, q4)

translated_skyrmion = translate_sk(my_skyrmion, [1.0,0.0,0.0]) 

display(plot_baryon_density(translated_skyrmion,iso_value=0.5))

In [ ]:
# we can even check the center of mass
center_of_mass(translated_skyrmion)

There are now two active skyrmions: my_skyrmion and translated_skyrmion. We can combine the skyrmions using the product approximation through the `product_approx` function. Let's try and make a twisted cube. This is made from two B=4 cubes, one rotated by 90 degrees with respect to the other.

To make this easier, let's start from the beginning. We're making a B=8 skyrmion, so let's use a slightly longer grid in the x-direction. First make the initial skyrmion and the translated skyrmion:

In [ ]:
my_skyrmion = Skyrmion([60,40,40],[0.2,0.2,0.2])
makeRM!(my_skyrmion, f4, p4, q4)
translated_skyrmion = translate_sk(my_skyrmion, [2.0,0.0,0.0]) 

# Then translate the original skyrmion in the other direction, and rotate it by pi/2 around the x-axis

rotate_sk!(my_skyrmion, pi/2, [1.0,0.0,0.0] )
translate_sk!(my_skyrmion, [-2.0,0.0,0.0])

# Having generated the two skyrmions we can combine them using the Product function

B8_product_skyrmion = product_approx(my_skyrmion, translated_skyrmion);

In [ ]:
# ... and plot it
display(plot_baryon_density(B8_product_skyrmion,iso_value=1.0))

The code features another way to do this, starting from Rational Maps. We can give a list of RM functions, positions, iso-orientations and orientations to the function make_RM_product!

Now that the skyrmions are getting more complicated, so is the notation. We can get some help using the help function. This works by sticking a `?` infront of the function

In [ ]:
?make_RM_product!

Hopefully, based on the help you can figure out how to use the function. Here's the code which generates the twisted B=8:

In [ ]:
X_list = [ (f4, p4, q4, [2.0,0.0,0.0], 0.0, [0.0,0.0,1.0], 0.0, [0.0,0.0,0.0] ),
    (f4, p4, q4, [-2.0,0.0,0.0], 0.0, [0.0,0.0,1.0], pi/2, [1.0,0.0,0.0] ) ]

make_RM_product!(my_skyrmion,X_list)

display(plot_baryon_density(my_skyrmion,iso_value=0.5))

This is a lot less hectic than the other method. Hopefully, you can see how this can be generalise in many many ways! Try making a B=12 triangle, or a B=108 crystal!

## ADHM construction

Let's now try making a Skyrmion from ADHM data. To do this we'll use `Makie`'s internal `Quaternion` type. Here, a quaternion is created using Quaternion(i,j,k,1). Note: the real part is the LAST component.

The `makeADHM!` function makes a skyrmion using `L` and `M`, a vector and matrix of quatenrions respectively. Here, we make a `B=2` torus. For some more ideas, you could have a look at this paper: https://arxiv.org/abs/2110.15190

In [ ]:
B=2

L = [ Quaternion(0.0,0.0,0.0,0.0) for a in 1:B ]
M = [ Quaternion(0.0,0.0,0.0,0.0) for a in 1:B, b in 1:B ]

L[1] = Quaternion(0.0, 0.0, 0.0, sqrt(2.0))
L[2] = Quaternion(0.0, 0.0, sqrt(2.0), 0.0)

M[1,1] = Quaternion(1.0, 0.0, 0.0, 0.0)
M[1,2] = Quaternion(0.0, 1.0, 0.0, 0.0)
M[2,1] = Quaternion(0.0, 1.0, 0.0, 0.0)
M[2,2] = Quaternion(-1.0, 0.0, 0.0, 0.0)

    
# The makeADHM! function is similar to the makeRM! function:

makeADHM!(my_skyrmion,L,M)

display(plot_baryon_density(my_skyrmion, iso_value=1.5))

Now you can easily, e.g., change the iso-orientation of the Skyrmion by left multiplying L with a quaternion

In [ ]:
for a in 1:2
    L[a] = Quaternion(0.0,sin(pi/4),0.0,cos(pi/4))*L[a]
end

makeADHM!(my_skyrmion,L,M)

display(plot_baryon_density(my_skyrmion, iso_value=1.5))

In [ ]:
# or we can make the entire skyrmion a bit bigger

for a in 1:2
    L[a] *= Quaternion(0.0,0.0,0.0,2.0)
    for b in 1:2
        M[a,b] *= Quaternion(0.0,0.0,0.0,2.0)
    end 
end

makeADHM!(my_skyrmion,L,M)

#which requires a much smaller iso_value...
display(plot_baryon_density(my_skyrmion, iso_value=0.1))

## Physics

Actual physics is crucial for skyrmion research. So we hope to include physics at the core of this package. Your physical units are stored in your `skyrmion` object. You can set them anytime:

In [ ]:
my_skyrmion.Fpi = 180
my_skyrmion.ee = 3.2
my_skyrmion.mpi = 1.0

Now that your skyrmion knows its physics, we can calculate the Energy in these units, by _turning physics on_.

In [ ]:
turn_on_physical!(my_skyrmion)
Energy(my_skyrmion)

Now the result it written alongside the units of the object. More examples:

In [ ]:
getMOI(my_skyrmion)

In [ ]:
rms_baryon(my_skyrmion)

Note that you can go back to dimensionless units by running `turn_off_physical!(my_skyrmion)`

Moments appear a lot in actual calculations, and are simple to calculate here:

In [ ]:
Energy(my_skyrmion, moment=2)

In [ ]:
println(getMOI(my_skyrmion,moment=2))

That's the end of this tutorial- you can now make your own notebook! Have fun, and please request features and report bugs to Chris.